# Projekt 2 - Steam Sale!

### Karim Kanji - IA20

## 1. Data Scraping och Data Munging

### Imports

In [318]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os.path


### Uppgiften

In [327]:
#Definierar listorna
gameListTitle = []
gameListRatingWord = []
gameListRatingPrecent = []
gameListSale = []
gameListPrice = []
gameListPriceOriginal = []
gameListYear = []
gameListSystemWin = []
gameListSystemLin = []
gameListSystemOsx = []
gameListTimeStamp = []

#Vi loopar för att få 5 sidor av innehåll
for i in range(1,6):
    
    #Definierar res. Res är variablen som sparas och innehåller informationen i linken dvs. Steam sales sidan
    res = requests.get("https://store.steampowered.com/search/?specials=1&page="+ str(i))
    
    #Definierar variabeln soup som innehåller parsead version av res (-> html)
    soup = bs(res.text, "html.parser")
    
    #Vi hittar alla div's med klassen responsive_search_name_combined och gör en for loop på basis av detta
    for games in soup.findAll("div", class_ = "responsive_search_name_combined"):

        #I kommande segmentet går jag egenom all data som krävdes för uppgiften. Principen är att appenda till 
        #respektive lista det element vi vill ha, t.ex. gameListTitle.append(games.find("span", class_ = "title").text)
        #lägger till till gameListTitle listan texten på elementet med taggen "span" och klassen "title" för att
        #skapa en lista med tittlar.
        gameListTitle.append(games.find("span", class_ = "title").text)
        
        #Definierar hur vi vill att datumet ser ut
        gameListTimeStamp.append(pd.to_datetime("today").strftime("%d/%m/%Y"))

        if (games.find("div", class_ = "search_review_summary positive") != 1):
            
            #Första gången vi hamnar splitta elementet vi hanterar för att endast extrahera den datan vi behöver, i detta fall splittar jag vid <br> och väljer
            #första elementet: -> 0 <- <br> 1
            if ((games.find(class_ = "col search_reviewscore responsive_secondrow").span) != None):
                gameListRatingWord.append(games.find(class_ = "col search_reviewscore responsive_secondrow").span["data-tooltip-html"].split('<br>')[0])
                
                #Här hamnar vi splitta två gånger för att få den datan vi vill ha
                gameListRatingPrecent.append(games.find(class_ = "col search_reviewscore responsive_secondrow").span["data-tooltip-html"].split('<br>')[1].split(" ")[3])
            else:
                gameListRatingWord.append("")
                gameListRatingPrecent.append("")
            
            
            #En if sats som kollar ifall span elementet finns, om det inte finns lägger den till listan en tom sträng
            #ifall det finns så lägger den till den datan vi vill ha
            if ((games.find(class_ = "col search_discount responsive_secondrow").span) == None):
                gameListSale.append("")
            else:
                gameListSale.append(games.find(class_ = "col search_discount responsive_secondrow").find("span").text)
            
            #Här väljer vi vilken del av texten vi vill ha med syntaxen: text[A:B]
            gameListPrice.append(games.find(class_ ="col search_price_discount_combined responsive_secondrow").text[15:21])
            gameListPriceOriginal.append(games.find(class_ ="col search_price_discount_combined responsive_secondrow").text[9:15])
            gameListYear.append(games.find(class_ ="col search_released responsive_secondrow").text[-4:])

            #Här börjar vi och kolla om spelet i fråga är kompatibel med olika operativsystem
        if (games.find("span", class_ ="platform_img win") == None ):
            gameListSystemWin.append("0")
        else:
            gameListSystemWin.append("1")

        if (games.find("span", class_ ="platform_img mac") == None ):
            gameListSystemOsx.append("0")
        else:
            gameListSystemOsx.append("1")

        if (games.find("span", class_ ="platform_img linux") == None ):
            gameListSystemLin.append("0")
        else:
            gameListSystemLin.append("1")



### Definierar den slutliga listan och använder .replace() för att få en vettig rating 

In [328]:
percentile_list = pd.DataFrame(
    {'Title': gameListTitle,
     'Rating 1-8': gameListRatingWord,
     'Reviews': gameListRatingPrecent,
     'Sale' : gameListSale,
     'Sale Price' : gameListPrice,
     'Original Price' : gameListPriceOriginal,
     'Year of release' : gameListYear,
     'Win' : gameListSystemWin,
     'Lin' : gameListSystemLin,
     'Osx' : gameListSystemOsx,
     'Info retreived': gameListTimeStamp
    })
percentile_list.replace(["Very Positive", "Overwhelmingly Positive", "Positive", "Mostly Positive",
                        "Mixed","Mostly Negative","Negative","Very Negative","Overwhelmingly Negative" ], ["8","9","7","6","5","4","3","2","1"])

,Title,Rating 1-8,Reviews,Sale,Sale Price,Original Price,Year of release,Win,Lin,Osx,Info retreived
0,"Warhammer 40,000: Inquisitor - Martyr",6,"13,693",-80%,"9,99€","49,99€",2018,1,0,0,29/11/2022
1,X4: Foundations,6,"12,254",-60%,"19,99€","49,99€",2018,1,1,0,29/11/2022
2,Battlefield™ 2042,4,"120,415",-67%,"19,79€","59,99€",2021,1,0,0,29/11/2022
3,X4 2022 Bundle,6,"12,978",-71%,"€39,14","134,92",,1,1,0,29/11/2022
4,"Warhammer 40,000: Inquisitor - Martyr - Sorori...",6,62,-10%,"13,49€","14,99€",2022,1,0,0,29/11/2022
...,...,...,...,...,...,...,...,...,...,...,...
120,Warhammer: Vermintide 2 - Starter Bundle,8,"71,401",,,,,1,0,0,29/11/2022
121,EVERSPACE™ - Ultimate Edition,6,"8,831",,,,,1,1,1,29/11/2022
122,Frostpunk: Season Pass,5,"2,798",,,,,1,0,1,29/11/2022
123,Best of Klei 2022,9,"438,391",,,,,1,0,0,29/11/2022


### Skapar csv filen ifall den inte finns, och om den finns så uppdaterar vi den

In [321]:
path = './SteamSales.csv'


if (os.path.exists(path)):
    percentile_list.to_csv("SteamSales.csv", mode = "a", index = False, header = False)
else:
    percentile_list.to_csv("SteamSales.csv", mode = "a", index = False)

### Använder head() för att få en bättre översikt av datan

In [322]:
percentile_list.head(10)

,Title,Rating 1-8,Reviews,Sale,Sale Price,Original Price,Year of release,Win,Lin,Osx,Info retreived
0,"Warhammer 40,000: Inquisitor - Martyr","74% of the 13,693 user reviews for this game a...","13,693",-80%,"9,99€","49,99€",2018,1,0,0,29/11/2022
1,X4: Foundations,"75% of the 12,254 user reviews for this game a...","12,254",-60%,"19,99€","49,99€",2018,1,1,0,29/11/2022
2,Battlefield™ 2042,"32% of the 120,415 user reviews for this game ...","120,415",-67%,"19,79€","59,99€",2021,1,0,0,29/11/2022
3,X4 2022 Bundle,"76% of the 12,978 user reviews for games in th...","12,978",-71%,"€39,14","134,92",,1,1,0,29/11/2022
4,"Warhammer 40,000: Inquisitor - Martyr - Sorori...",70% of the 62 user reviews for this game are p...,62,-10%,"13,49€","14,99€",2022,1,0,0,29/11/2022
5,No Man's Sky,"74% of the 195,932 user reviews for this game ...","195,932",-50%,"27,49€","54,99€",2016,1,0,0,29/11/2022
6,SCUM,"75% of the 65,545 user reviews for this game a...","65,545",-50%,"14,99€","29,99€",2018,1,0,0,29/11/2022
7,Call of the Wild: The Angler™,"52% of the 3,150 user reviews for this game ar...","3,150",-34%,"19,79€","29,99€",2022,1,0,0,29/11/2022
8,Divinity: Original Sin - The Source Saga,"95% of the 151,759 user reviews for games in t...","151,759",,,,,1,0,1,29/11/2022
9,Barotrauma,"93% of the 26,180 user reviews for this game a...","26,180",-75%,"6,24€","24,99€",2019,1,1,1,29/11/2022


### Använder describe() för att få statistik över datan som t.ex. mängden (count) och topp spelet

In [323]:
percentile_list.describe()

,Title,Rating 1-8,Reviews,Sale,Sale Price,Original Price,Year of release,Win,Lin,Osx,Info retreived
count,125,125,125,125,125,125,125,125,125,125,125
unique,125,124,124,19,28,20,10,1,2,2,1
top,"Warhammer 40,000: Inquisitor - Martyr",,,,,,,1,0,0,29/11/2022
freq,1,2,2,89,89,89,93,125,84,83,125
